&nbsp;

# 21 - Implémentation du modèle SINDy

---

&nbsp;

> ### Qu'est-ce que la méthode SINDy et comment l'implémenter pour notre jeu de donnée ?

La méthode SINDy (ou *Sparse Identification of Non-linear Dynamic*) cherche à représenter l'évolution temporelle d'un état $x(t)$ sous la forme d'une fonction non-linéaire $f$, soit :

$$
\frac{d}{dt}x(t) = f(x(t))
$$

Exactement la forme que l'on cherche à approximer. On admet que cette équation consistue un système dynamique pour mesurer $x(t)$, où l'état est en fait un vecteur comme suit :

$$
x(t) = [x_1(t), x_2(t). \dots, x_n(t)]^{\perp} 
$$

On peut aisément identifier que $x(t)$ correspond bien à notre état réduit actuel déjà préparé. La fonction $f(x(t))$ vient contraindre comment le système évolue dans le temps. L'idée clé derrière SINDy est que la fonction est souvent répandue dans l'espace d'un ensemble approprié de fonctions de base. On pourrait donner l'exemple suivant :

$$
\frac{d}{dt}x(t) = f(x) = 
\begin{bmatrix}
   f_1(x) \\
   f_2(x)
\end{bmatrix} = 
\begin{bmatrix}
   1 - x_1²x_2 + 8x_1 \\
   1 - 4x_1³
\end{bmatrix}
$$

On dit que la fonction est *parcimonieuse*(ou *sparse*) car la descriptivité est approximée en *sets* de polynômes de deux variables d'une manière que si on écrivait nos fonctions de la forme d'une expansion de fonctions composantes $f_y(x) = \sum^{\infty}_{i=0}\sum^{\infty}_{j=0} a_{i,j}x_1^i x_2^j$, alors il y aurait très peu de coefficients $a_{i,j}$ qui serait non-nuls. 

SINDy emploie une régression parcimonieuse pour chercher une combinaison linéaire de fonctions de base qui capture au mieux la dynamique comporementale du système physique.

Maintenant que nous avons connaissance des fondamentaux de la méthode, démarrons l'implémentation par les basiques.

In [5]:
# We import all libs we will need to implement properly SINDy method
using NPZ
using LinearAlgebra
using Statistics
using ModelingToolkit
using OrdinaryDiffEq
using Printf
using Statistics
using Plots

# We import our reduced state (build by selection)
data = npzread("data/processed/sstReducedState2COPERNICUS20102019.npz")

# We extract main parameters from it
Z  = Float64.(data["Z"]) #(time, state)
dZ = Float64.(data["dZ"]) # We cast as Float64 to avoid futures errors of type with lib imported functions
splitMask = Int.(data["split"])

# We permut to a preferrable order
Z  = permutedims(Z) #(state, time) 
dZ = permutedims(dZ)

nState, T = size(Z) # Thanks to size(x) function we can extract bounds

# We seek for bounds indexes to delimit test and train sets
# split is a shape of (0, 0, 0, 0, ..., 1, 1, 1)
trainIdx = findall(splitMask .== 0) 
testIdx  = findall(splitMask .== 1)

# We properly define to work easily with our state later
ZTrain  = Z[:, trainIdx]
dZTrain = dZ[:, trainIdx]
ZTest   = Z[:, testIdx]
dZTest  = dZ[:, testIdx];

Ensuite, par rigueur, nous allons normaliser de nouveau pour éviter toute sparsité biaisé (étant donné que nous travaillerons avec un seuil, nous voulons que les coefficients soient sur un pied d'égalité).

In [7]:
# we normalize again "state by state" to ensure sparse regression will operate correctly (as state-wise)
# of course we compute mean and standard error only on train set to avoid any anticipation biais
μ = mean(ZTrain; dims=2)
σ = std(ZTrain; dims=2) .+ 1e-8

ZTrainN = (ZTrain .- μ) ./ σ
ZTestN  = (ZTest  .- μ) ./ σ

dZTrainN = dZTrain ./ σ
dZTestN  = dZTest  ./ σ;

Entrons désormais dans le coeur de la méthode SINDy.

En l'état, nous avons pour le moment les deux matrices suivantes (aggrégées par nos données): 

$$
X = \begin{bmatrix}
   x_1(t_1) & x_2(t_1) & \dots & x_n(t_1) \\
   x_1(t_2) & x_2(t_2) & \dots & x_n(t_2) \\
   \vdots & \vdots & & \vdots \\
   x_1(t_m) & x_2(t_m) & \dots & x_n(t_m) \\
\end{bmatrix}, \space \space \space \dot{X} = \begin{bmatrix}
   \dot{x_1(t_1)} & \dot{x_2(t_1)} & \dots & \dot{x_n(t_1)} \\
   \dot{x_1(t_2)} & \dot{x_2(t_2)} & \dots & \dot{x_n(t_2)} \\
   \vdots & \vdots & & \vdots \\
   \dot{x_1(t_m)} & \dot{x_2(t_m)} & \dots & \dot{x_n(t_m)} \\
\end{bmatrix}
$$

L'hypothèse clé de la méthode est que le champ de vecteurs est parcimonieux dans une base de fonctions appropriée (l'hypothèse est physique, pas statistique, on imagine décrire le système par des lois simples). Mathématiquement cela correspond à

$$f(x) \in \text{span}[\theta_1(x), \theta_2(x), \dots, \theta_p(x)]$$

mais avec peu de termes actifs. Autrement dit :

$$
\dot{x}(t) = \overset{p}{\underset{j=1}{\sum}} \xi_j \theta_j(x(t)), \space ||\xi||_0 \ll p
$$

&nbsp;

In [9]:
# We declare symbolic variables (nState times so it match dimensions correctly)
@variables x[1:nState]

# We build the local polynomial lib
basis = Num[]
push!(basis, 1) # we start by pushing 1

#then we push several ways to arrange our symbolic variables

# linearly
for i in 1:nState
    push!(basis, x[i])
end

# quadraticly local
for i in 1:nState
    push!(basis, x[i]^2)
    if i < nState
        push!(basis, x[i] * x[i+1])
    end
end

nbasis = length(basis)
println("Reduced library size = ", nbasis)
println("Lib :", basis)

Reduced library size = 51
Lib :Num[1, x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12], x[13], x[14], x[15], x[16], x[17], x[1]^2, x[1]*x[2], x[2]^2, x[2]*x[3], x[3]^2, x[3]*x[4], x[4]^2, x[4]*x[5], x[5]^2, x[5]*x[6], x[6]^2, x[6]*x[7], x[7]^2, x[7]*x[8], x[8]^2, x[8]*x[9], x[9]^2, x[10]*x[9], x[10]^2, x[10]*x[11], x[11]^2, x[11]*x[12], x[12]^2, x[12]*x[13], x[13]^2, x[13]*x[14], x[14]^2, x[14]*x[15], x[15]^2, x[15]*x[16], x[16]^2, x[16]*x[17], x[17]^2]


Pour cela, nous devons définir une bibliothèque de candidats $\Theta(x)$ de la forme

$$
\Theta(x) = [\theta_1(x) \space \theta_2(x) \space \dots \space \theta_p(x)]
$$

où chaque terme peut être une constante, un état clé, un couplage d'états, et plus. Pour chaque instant $t_k$, on évalue : 

$$
\Theta(x(t_k)) \in \mathbb{R}^p
$$

In [13]:
# we build Θ function
# ModelingToolkitBase permit us to perform symbolical-numeric computation, it is unevitable when sparsifying or parallelizing
# See following link for more details about the lib : https://juliapackages.com/p/modelingtoolkit

# Here we initialize the 0 function using our candidates dict
ΘBase = ModelingToolkit.build_function( #build_function method can be hard to find in documentation because it is low level API in the lib but it just compile a function for Julia without symbolic and very fastly
    basis, x;
    expression = Val(false)
)[1]

# Then we build Θ matrix
function buildΘ(Zmat, ΘBase, nbasis)
    Tloc = size(Zmat, 2) # we search for the time size of the matrix here
    Θ = zeros(Float64, Tloc, nbasis) # we preform a matrix of the right shape
    for t in 1:Tloc
        Θ[t, :] .= ΘBase(Zmat[:, t]) # we just apply 0 function to state matrix 
    end
    return Θ
end

ΘTrain = buildΘ(ZTrainN, ΘBase, nbasis)
ΘTest  = buildΘ(ZTestN,  ΘBase, nbasis)

println("ΘTrain :", ΘTrain)

ΘTrain :[1.0 -0.027307412431927452 0.1650844965528944 0.1349328597207983 -0.12966407428655888 0.21687781399185335 0.05749625098307566 0.27702741092188354 0.21816559265232743 0.18694745351062145 0.23029478644074403 -0.10469897647710996 -0.22626175125870973 0.08405386135467867 0.24984321101581025 -0.17161771374967336 0.01651266978400426 0.2116468547492219 0.000745694773727386 -0.004508030433486993 0.027252891002122602 0.02227532321545031 0.01820687663243263 -0.017495944346535418 0.01681277216059026 -0.028121260984546172 0.04703598620188494 0.012469661225936398 0.003305818877108829 0.015928037547556253 0.07674418640208212 0.06043784928471257 0.04759622581734126 0.040785501989988156 0.034949350374105966 0.04305302388186949 0.0530356886617879 -0.02411162842836052 0.010961875675354425 0.023689373772705354 0.05119438008265823 -0.01901817387016638 0.007065051608631545 0.021000286619130643 0.06242163009069069 -0.04287752067041056 0.029452639672664822 -0.002833866636234124 0.00027266826339556735

Excessive output truncated after 524300 bytes.

 0.07927828101401763 0.03400111930568653 0.014582507330540092 0.029596605500574146 0.06006916624839256 0.06156600193205496 0.06310013656963694; 1.0 -0.10765069265222328 0.006134513336270573 -0.2461365944730335 -0.04312685335222805 -0.2789811522646335 -0.12032205332961225 -0.3509296093923493 -0.22597596708515955 -0.33539805359886016 -0.330046990773402 -0.2391154734395647 -0.07774362309650673 -0.2397224813259711 -0.29060533172525005 -0.08513761408456072 -0.22755253257213912 -0.23323990218915702 0.01158867162850344 -0.0006603846097338283 3.763225387288152e-5 -0.0015099282213390458 0.06058322313878254 0.010615096814455341 0.001859925480064584 0.012031579241752454 0.07783048331890262 0.03356758508074191 0.014477396517454055 0.04222457117624625 0.12315159074826686 0.07930165786125343 0.05106513770007311 0.0757918995204826 0.11249185435790388 0.11069711830155998 0.1089310161185781 0.07891934245608566 0.05717620963822718 0.018589703243628283 0.006044070932171695 0.01863689423596567 0.057466868